In [2]:
import pandas as pd
from itertools import combinations

In [ ]:
# get only the utility patents from g_patent
g_patent = pd.read_csv("g_patent.csv")
g_patent = g_patent[g_patent["patent_type"] == "utility"]
g_patent.to_csv("g_patent_utility.csv")

In [13]:
# merge “patent_id” from g_us_patent_citation with “patent_id” from g_patent to get the “patent_date”
# the tables take up a lot of RAM, thus we will drop unneeded columns to save memory
g_patent_utility = pd.read_csv("g_patent_utility.csv")
g_patent_utility_min = g_patent_utility[["patent_id", "patent_date"]]
g_patent_utility_min.to_csv("g_patent_utility_min.csv")

g_us_patent = pd.read_csv("g_us_patent_citation.csv")
g_us_patent = g_us_patent[["patent_id", "citation_patent_id", "citation_date"]]
g_us_patent.to_csv("g_us_patent_min.csv")

g_cpc_current = pd.read_csv("g_cpc_current.csv")
g_cpc_current = g_cpc_current[["patent_id", "cpc_subclass"]]
g_cpc_current.to_csv("g_cpc_current_min.csv")

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_2414/3649613640.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  g_patent_utility = pd.read_csv("g_patent_utility.csv")


In [2]:
# read all the minimal csv files
g_us_patent_min = pd.read_csv("g_us_patent_min.csv")
g_patent_utility_min = pd.read_csv("g_patent_utility_min.csv")

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51109/252669266.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  g_us_patent_min = pd.read_csv("g_us_patent_min.csv")
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51109/252669266.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  g_patent_utility_min = pd.read_csv("g_patent_utility_min.csv")


In [49]:
# merged utility patents with cited patent
merged_df = g_patent_utility_min.merge(g_us_patent_min, on="patent_id", how="inner")

# all the citations from g_cpc_current are numeric type while "citation_patent_id" is object type in this table
merged_df["citation_patent_id"] = pd.to_numeric(merged_df["citation_patent_id"], errors="coerce")
merged_df["gyear"] = merged_df["patent_date"].astype(str).str[:4]
merged_df.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y", "patent_date", "citation_date"], inplace=True)
merged_df.drop_duplicates(inplace=True)
merged_df.dropna(inplace=True)

# remove the patents that only cite from another patent once (no pair of patents cited to make a comparison)
value_counts = merged_df.value_counts("patent_id")
single_appearance_patents = value_counts[value_counts == 1].index.tolist()
merged_df = merged_df[~merged_df['patent_id'].isin(single_appearance_patents)]
merged_df
#merged_df.to_csv("merged_df.csv")

NameError: name 'g_patent_utility_min' is not defined

In [50]:
# cleaning cpc class data to merge
g_cpc_current_min = pd.read_csv("g_cpc_current_min.csv")
g_cpc_current_min.rename(columns={"patent_id":"citation_pair_1"}, inplace=True)
g_cpc_current_min.drop(columns="Unnamed: 0", inplace=True)
g_cpc_current_min.drop_duplicates(inplace=True)
g_cpc_current_min

,citation_pair_1,cpc_subclass
0,5664589,A45D
1,10439720,G02B
2,9840937,F01D
3,7407213,B62D
4,11014297,B29C
...,...,...
50036463,9797812,G01L
50036467,10727269,H01L
50036472,9638814,G01T
50036473,10964018,G16H


In [3]:
merged_df = pd.read_csv("merged_df.csv")
merged_df.drop(columns="Unnamed: 0", inplace=True)
merged_df 

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_54112/1255860723.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("merged_df.csv")


,patent_id,citation_patent_id,gyear
0,10000000,5093563.0,2018
1,10000000,5751830.0,2018
2,10000001,7804268.0,2018
3,10000001,9022767.0,2018
4,10000001,9090016.0,2018
...,...,...,...
114602467,RE34947,4185894.0,1995
114602468,RE34947,4368386.0,1995
114602469,RE34947,4481531.0,1995
114602470,RE34947,4920417.0,1995


In [4]:
# The following analysis is based on the sub-sample of patents with only 2 cites.
patent_count = merged_df["patent_id"].value_counts()
# patents with 2 cites
patents1 = patent_count[patent_count <= 2].index
merged_df1 = merged_df[merged_df['patent_id'].isin(patents1)]

merged_df1

,patent_id,citation_patent_id,gyear
0,10000000,5093563.0,2018
1,10000000,5751830.0,2018
15,10000004,5632133.0,2018
16,10000004,6726363.0,2018
171,10000019,5582677.0,2018
...,...,...,...
114601716,9999975,5548694.0,2018
114602157,9999994,4935175.0,2018
114602158,9999994,9052444.0,2018
114602183,9999999,4579520.0,2018


In [53]:
# for each patent_id, create a column with a list of all the citation_patent_id
grouped = merged_df1.groupby('patent_id')['citation_patent_id'].apply(list).reset_index(name='citation_pairs')

result_rows = []

for i, row in grouped.iterrows():
    citation_pairs = row['citation_pairs']
    
    # create every possible pairing for each row
    pairs = list(combinations(citation_pairs, 2))
    
    # add each pair to our list
    result_rows.extend([(row['patent_id'], pair[0], pair[1]) for pair in pairs])

# Create the result DataFrame
result_df = pd.DataFrame(result_rows, columns=['patent_id', 'citation_pair_1', 'citation_pair_2'])

result_df.drop_duplicates(inplace=True)
result_df = result_df[result_df["citation_pair_1"] != result_df["citation_pair_2"]]
#result_df.to_csv("result_2.csv")
result_df

,patent_id,citation_pair_1,citation_pair_2
0,3930271,2379430.0,2782422.0
1,3930274,2546396.0,3064370.0
2,3930274,2546396.0,3601076.0
3,3930274,3064370.0,3601076.0
4,3930275,914377.0,2971278.0
...,...,...,...
11608095,RE31701,3408931.0,3911817.0
11608096,RE31701,3408931.0,4057452.0
11608097,RE31701,3490365.0,3911817.0
11608098,RE31701,3490365.0,4057452.0


In [3]:
# among all steps, save the merged table up to date to csv file so we can read directly from it rather than running all the code again to build it
result_df = pd.read_csv("result_2.csv")
result_df.drop(columns="Unnamed: 0", inplace=True)
result_df

,patent_id,citation_pair_1,citation_pair_2
0,3930271,2379430.0,2782422.0
1,3930276,3037223.0,3554132.0
2,3930284,3461482.0,3599277.0
3,3930292,3172187.0,3323191.0
4,3930299,3207406.0,3716179.0
...,...,...,...
643350,9999942,5429810.0,6380149.0
643351,9999956,6390908.0,6923709.0
643352,9999975,5347459.0,5548694.0
643353,9999994,4935175.0,9052444.0


In [54]:
# the merge is done at cpc level, each patent citation pair can have multiple rows depending on the number of cpcs they have.
# merge with cpc classes
cpc_df = result_df.merge(g_cpc_current_min, on="citation_pair_1", how="inner")
#cpc_df.drop(columns="citation_pair_1", inplace=True)
g_cpc_current_min.rename(columns={"citation_pair_1": "citation_pair_2"}, inplace=True)
cpc_df_merge = cpc_df.merge(g_cpc_current_min, on="citation_pair_2", how="inner")
cpc_df_merge

KeyboardInterrupt: 

: 

In [6]:
# merge with gyear
cpc_df_year = cpc_df_merge.merge(merged_df, on="patent_id", how="inner")
cpc_df_year.drop(columns="citation_patent_id", inplace=True)
cpc_df_year.drop_duplicates(inplace=True)

# count occurrences of x and y
cpc_df_year["cpc_count_x"] = cpc_df_year.groupby(['cpc_subclass_x','gyear'])['cpc_subclass_x'].transform("count")
cpc_df_year["cpc_count_y"] = cpc_df_year.groupby(['cpc_subclass_y','gyear'])['cpc_subclass_y'].transform("count")

# find how many patents cite both x and y classes (cited pairs)
cpc_df_year["citation_pair_count"] = cpc_df_year.groupby(['cpc_subclass_x', 'cpc_subclass_y', 'gyear'])["patent_id"].transform("count")

cpc_df_year[(cpc_df_year["cpc_subclass_x"] == "H01H")&((cpc_df_year["gyear"] <= 1978))]

,patent_id,citation_pair_1,citation_pair_2,cpc_subclass_x,cpc_subclass_y,gyear,cpc_count_x,cpc_count_y,citation_pair_count
0,3988554,3940583.0,3941962.0,H01H,H01H,1976,1,1,1
2,4079218,3940583.0,3975602.0,H01H,H01H,1978,7,5,5
1956,4071835,3974468.0,3979704.0,H01H,H01H,1978,7,5,5
2150,4075587,3942142.0,3953813.0,H01H,H01H,1978,7,5,5
3094,4086549,4001652.0,4034266.0,H01H,H01H,1978,7,5,5
3096,4086549,4001652.0,4034266.0,H01H,H01R,1978,7,3,1
6156,4118681,4013984.0,4025883.0,H01H,H01H,1978,7,5,5
6734,4127757,4016377.0,4025747.0,H01H,H02B,1978,7,1,1


In [7]:
# created a new table on the cpc-pair-year level
cumulative_year_df = cpc_df_year[["cpc_subclass_x", "cpc_subclass_y", "gyear", "citation_pair_count"]]
cumulative_year_df #2422864 -> 525284
cumulative_year_df.drop_duplicates(inplace=True)

# get the culmulative pair occurrence by year
cumulative_year_df.sort_values(by="gyear", inplace=True)
cumulative_year_df["cumulative_pair_count"] = cumulative_year_df.groupby(['cpc_subclass_x', 'cpc_subclass_y'])['citation_pair_count'].cumsum()

cumulative_year_df

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/1107616374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_year_df.drop_duplicates(inplace=True)
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/1107616374.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_year_df.sort_values(by="gyear", inplace=True)
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/1107616374.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,cpc_subclass_x,cpc_subclass_y,gyear,citation_pair_count,cumulative_pair_count
0,H01H,H01H,1976,1,1
4,C10M,C07C,1976,1,1
6,C10M,C08F,1976,1,1
8,C07C,C07C,1976,1,1
10,C07C,C08F,1976,1,1
...,...,...,...,...,...
3549130,G09F,G06T,2023,1,19
3549132,G09F,H04N,2023,1,90
1842224,H10N,H10N,2023,20,1551
373920,Y10S,C12N,2023,11,2278


In [8]:
# created a new table on the cpc-x-subclass and year level
cumulative_x_df = cpc_df_year[["cpc_subclass_x", "gyear", "cpc_count_x"]]
cumulative_x_df.drop_duplicates(inplace=True)

# get the culmulative x occurrence by year
cumulative_x_df.sort_values(by="gyear", inplace=True)
cumulative_x_df["cumulative_x_count"] = cumulative_x_df.groupby('cpc_subclass_x')['cpc_count_x'].cumsum()
cumulative_x_df.rename(columns={"cpc_subclass_x": "cpc_subclass"}, inplace=True)
cumulative_x_df[(cumulative_x_df["cpc_subclass"] == "H01H")]

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/2988798584.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_x_df.drop_duplicates(inplace=True)
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/2988798584.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_x_df.sort_values(by="gyear", inplace=True)
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/2988798584.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,cpc_subclass,gyear,cpc_count_x,cumulative_x_count
0,H01H,1976,1,1
2,H01H,1978,7,8
7796,H01H,1979,8,16
14988,H01H,1980,16,32
20972,H01H,1981,27,59
22636,H01H,1982,33,92
37650,H01H,1983,17,109
33042,H01H,1984,34,143
53886,H01H,1985,39,182
33048,H01H,1986,43,225


In [9]:
# created a new table on the cpc-y-subclass and year level
cumulative_y_df = cpc_df_year[["cpc_subclass_y", "gyear", "cpc_count_y"]]
cumulative_y_df.drop_duplicates(inplace=True)

# get the culmulative x occurrence by year
cumulative_y_df.sort_values(by="gyear", inplace=True)
cumulative_y_df["cumulative_y_count"] = cumulative_y_df.groupby('cpc_subclass_y')['cpc_count_y'].cumsum()
cumulative_y_df.rename(columns={"cpc_subclass_y": "cpc_subclass"}, inplace=True)
cumulative_y_df[(cumulative_y_df["cpc_subclass"] == "H01H")]


/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/137821017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_y_df.drop_duplicates(inplace=True)
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/137821017.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_y_df.sort_values(by="gyear", inplace=True)
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/137821017.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,cpc_subclass,gyear,cpc_count_y,cumulative_y_count
0,H01H,1976,1,1
2,H01H,1978,5,6
7796,H01H,1979,10,16
14988,H01H,1980,16,32
20972,H01H,1981,25,57
22636,H01H,1982,39,96
37650,H01H,1983,24,120
15732,H01H,1984,41,161
46168,H01H,1985,46,207
33048,H01H,1986,44,251


In [14]:
# combine cumulative x and y to get the sum of total occurrences of each cpc_subclass
cumulative_x_y = cumulative_x_df.merge(cumulative_y_df, on=["cpc_subclass","gyear"], how="inner")
cumulative_x_y["cpc_count_by_year"] = cumulative_x_y["cpc_count_x"] + cumulative_x_y["cpc_count_y"]
cumulative_x_y["cumulative_total_count"] = cumulative_x_y["cumulative_x_count"] + cumulative_x_y["cumulative_y_count"]

# rename column to merge with main table 
cumulative_x_y.rename(columns={"cpc_subclass": "cpc_subclass_x"}, inplace=True)

# drop now unneccessary columns
cumulative_x_y.drop(columns=["cumulative_y_count","cumulative_x_count"], inplace=True)
cumulative_x_y.drop(columns=["cpc_count_x","cpc_count_y"], inplace=True)
cumulative_x_y

,cpc_subclass_x,gyear,cpc_count_by_year,cumulative_total_count
0,H01H,1976,2,2
1,C07C,1976,4,4
2,C09K,1977,9,9
3,C09D,1977,11,11
4,C08L,1977,11,11
...,...,...,...,...
22033,C08J,2023,242,14980
22034,C08G,2023,262,30842
22035,G01T,2023,42,4020
22036,Y02P,2023,950,45052


In [15]:
# map cpc_count_total to the x and y values of the df

# map total x count by year to cpc_subclass_x
cpc_counts_by_year = cpc_df_year.merge(cumulative_x_y, on=["cpc_subclass_x", "gyear"], how="inner")
cpc_counts_by_year.rename(columns={"cpc_count_by_year": "cpc_count_by_year_x"}, inplace=True)
cumulative_x_y.rename(columns={"cpc_subclass_x": "cpc_subclass_y"}, inplace=True)

# map total y count by year to cpc_subclass_y
both_cpc_counts = cpc_counts_by_year.merge(cumulative_x_y, on=["cpc_subclass_y", "gyear"], how="inner")
both_cpc_counts.rename(columns={"cpc_count_by_year": "cpc_count_by_year_y"}, inplace=True)
both_cpc_counts["either_x_or_y_by_year"] = both_cpc_counts["cpc_count_by_year_x"] + both_cpc_counts["cpc_count_by_year_y"]
both_cpc_counts["either_x_or_y_cumulative"] = both_cpc_counts["cumulative_total_count_x"] + both_cpc_counts["cumulative_total_count_y"]

both_cpc_counts

,patent_id,citation_pair_1,citation_pair_2,cpc_subclass_x,cpc_subclass_y,gyear,cpc_count_x,cpc_count_y,citation_pair_count,cpc_count_by_year_x,cumulative_total_count_x,cpc_count_by_year_y,cumulative_total_count_y,either_x_or_y_by_year,either_x_or_y_cumulative
0,3988554,3940583.0,3941962.0,H01H,H01H,1976,1,1,1,2,2,2,2,4,4
1,4079218,3940583.0,3975602.0,H01H,H01H,1978,7,5,5,12,14,12,14,24,28
2,4071835,3974468.0,3979704.0,H01H,H01H,1978,7,5,5,12,14,12,14,24,28
3,4075587,3942142.0,3953813.0,H01H,H01H,1978,7,5,5,12,14,12,14,24,28
4,4086549,4001652.0,4034266.0,H01H,H01H,1978,7,5,5,12,14,12,14,24,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412517,11602915,4041849.0,4557715.0,B31B,B31B,2023,5,1,1,6,495,6,495,12,990
2412518,11597022,11077509.0,11344961.0,B23G,B23G,2023,3,1,1,4,109,4,109,8,218
2412519,11597414,9352757.0,10723371.0,B61F,B61F,2023,3,1,1,4,226,4,226,8,452
2412520,11603760,4052859.0,9890637.0,E21D,E21D,2023,2,1,1,3,290,3,290,6,580


In [19]:
# merge with cumulative count with cpc x and y subclasses
final_df = both_cpc_counts.merge(cumulative_year_df, on=["cpc_subclass_x", "cpc_subclass_y", "gyear", "citation_pair_count"], how="inner")
final_df.rename(columns={"citation_pair_count": "citation_pair_count_by_year"}, inplace=True)
final_df.drop(columns=["cpc_count_x", "cpc_count_y"], inplace=True)

final_df[final_df["cpc_subclass_x"] != final_df["cpc_subclass_y"]]

,patent_id,citation_pair_1,citation_pair_2,cpc_subclass_x,cpc_subclass_y,gyear,citation_pair_count_by_year,cpc_count_by_year_x,cumulative_total_count_x,cpc_count_by_year_y,cumulative_total_count_y,either_x_or_y_by_year,either_x_or_y_cumulative,cumulative_pair_count
6,4086549,4001652.0,4034266.0,H01H,H01R,1978,1,12,14,8,17,20,31,1
8,4126371,4064412.0,4075531.0,H01J,H01R,1978,1,23,35,8,17,31,52,1
19,4003946,3932479.0,3948981.0,C07F,C07C,1977,2,24,24,47,51,71,75,2
20,4060691,3950406.0,3965143.0,C07F,C07C,1977,2,24,24,47,51,71,75,2
21,4003946,3932479.0,3948981.0,C07D,C07C,1977,5,88,88,47,51,135,139,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412498,11604054,4869171.0,8967048.0,F42D,F42C,2023,1,8,133,7,166,15,299,4
2412499,11604054,4869171.0,8967048.0,F42B,F42C,2023,1,32,946,7,166,39,1112,7
2412502,11472318,4378946.0,5553914.0,B62B,A47D,2022,1,97,837,6,193,103,1030,3
2412508,11543132,6938424.0,7779637.0,F23R,F23M,2023,1,25,1011,4,296,29,1307,12


In [20]:
# reorder column
final_df = final_df[["patent_id","citation_pair_1", "citation_pair_2","gyear","cpc_subclass_x","cpc_subclass_y","citation_pair_count_by_year",	"cpc_count_by_year_x",	"cpc_count_by_year_y", "either_x_or_y_by_year", "cumulative_pair_count","cumulative_total_count_x",	"cumulative_total_count_y", "either_x_or_y_cumulative"]]
final_df

,patent_id,citation_pair_1,citation_pair_2,gyear,cpc_subclass_x,cpc_subclass_y,citation_pair_count_by_year,cpc_count_by_year_x,cpc_count_by_year_y,either_x_or_y_by_year,cumulative_pair_count,cumulative_total_count_x,cumulative_total_count_y,either_x_or_y_cumulative
0,3988554,3940583.0,3941962.0,1976,H01H,H01H,1,2,2,4,1,2,2,4
1,4079218,3940583.0,3975602.0,1978,H01H,H01H,5,12,12,24,6,14,14,28
2,4071835,3974468.0,3979704.0,1978,H01H,H01H,5,12,12,24,6,14,14,28
3,4075587,3942142.0,3953813.0,1978,H01H,H01H,5,12,12,24,6,14,14,28
4,4086549,4001652.0,4034266.0,1978,H01H,H01H,5,12,12,24,6,14,14,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412517,11602915,4041849.0,4557715.0,2023,B31B,B31B,1,6,6,12,24,495,495,990
2412518,11597022,11077509.0,11344961.0,2023,B23G,B23G,1,4,4,8,7,109,109,218
2412519,11597414,9352757.0,10723371.0,2023,B61F,B61F,1,4,4,8,32,226,226,452
2412520,11603760,4052859.0,9890637.0,2023,E21D,E21D,1,3,3,6,48,290,290,580


In [25]:
# calculate the technical propinquity of each pair that is not paired with themselves
final_df_no_duplicates = final_df[final_df["cpc_subclass_x"] != final_df["cpc_subclass_y"]]
final_df_no_duplicates["tech_prop"] = 1 - (final_df_no_duplicates["cumulative_pair_count"]/final_df_no_duplicates["either_x_or_y_cumulative"])
final_df_no_duplicates["revised_tech_prop"] = final_df_no_duplicates["tech_prop"]
final_df_no_duplicates

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/3335255728.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_no_duplicates["tech_prop"] = 1 - (final_df_no_duplicates["cumulative_pair_count"]/final_df_no_duplicates["either_x_or_y_cumulative"])
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/3335255728.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_no_duplicates["revised_tech_prop"] = final_df_no_duplicates["tech_prop"]


,patent_id,citation_pair_1,citation_pair_2,gyear,cpc_subclass_x,cpc_subclass_y,citation_pair_count_by_year,cpc_count_by_year_x,cpc_count_by_year_y,either_x_or_y_by_year,cumulative_pair_count,cumulative_total_count_x,cumulative_total_count_y,either_x_or_y_cumulative,tech_prop,revised_tech_prop
6,4086549,4001652.0,4034266.0,1978,H01H,H01R,1,12,8,20,1,14,17,31,0.967742,0.967742
8,4126371,4064412.0,4075531.0,1978,H01J,H01R,1,23,8,31,1,35,17,52,0.980769,0.980769
19,4003946,3932479.0,3948981.0,1977,C07F,C07C,2,24,47,71,2,24,51,75,0.973333,0.973333
20,4060691,3950406.0,3965143.0,1977,C07F,C07C,2,24,47,71,2,24,51,75,0.973333,0.973333
21,4003946,3932479.0,3948981.0,1977,C07D,C07C,5,88,47,135,5,88,51,139,0.964029,0.964029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412498,11604054,4869171.0,8967048.0,2023,F42D,F42C,1,8,7,15,4,133,166,299,0.986622,0.986622
2412499,11604054,4869171.0,8967048.0,2023,F42B,F42C,1,32,7,39,7,946,166,1112,0.993705,0.993705
2412502,11472318,4378946.0,5553914.0,2022,B62B,A47D,1,97,6,103,3,837,193,1030,0.997087,0.997087
2412508,11543132,6938424.0,7779637.0,2023,F23R,F23M,1,25,4,29,12,1011,296,1307,0.990819,0.990819


In [45]:
# isolate duplicate table and calculate tech prop
final_df_duplicates = final_df[final_df["cpc_subclass_x"] == final_df["cpc_subclass_y"]]
final_df_duplicates["tech_prop"] = 1 - (final_df_duplicates["cumulative_pair_count"]/final_df_duplicates["either_x_or_y_cumulative"])

# add another called revised_tech_prop and force value to be 1
final_df_duplicates["revised_tech_prop"] = 1

final_df_duplicates

/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/1227610290.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_duplicates["tech_prop"] = 1 - (final_df_duplicates["cumulative_pair_count"]/final_df_duplicates["either_x_or_y_cumulative"])
/var/folders/49/pdjkhlfj0mz7g524_6j5vd5w0000gn/T/ipykernel_51240/1227610290.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_duplicates["revised_tech_prop"] = 1


,patent_id,citation_pair_1,citation_pair_2,gyear,cpc_subclass_x,cpc_subclass_y,citation_pair_count_by_year,cpc_count_by_year_x,cpc_count_by_year_y,either_x_or_y_by_year,cumulative_pair_count,cumulative_total_count_x,cumulative_total_count_y,either_x_or_y_cumulative,tech_prop,revised_tech_prop
0,3988554,3940583.0,3941962.0,1976,H01H,H01H,1,2,2,4,1,2,2,4,0.750000,1
1,4079218,3940583.0,3975602.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1
2,4071835,3974468.0,3979704.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1
3,4075587,3942142.0,3953813.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1
4,4086549,4001652.0,4034266.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412517,11602915,4041849.0,4557715.0,2023,B31B,B31B,1,6,6,12,24,495,495,990,0.975758,1
2412518,11597022,11077509.0,11344961.0,2023,B23G,B23G,1,4,4,8,7,109,109,218,0.967890,1
2412519,11597414,9352757.0,10723371.0,2023,B61F,B61F,1,4,4,8,32,226,226,452,0.929204,1
2412520,11603760,4052859.0,9890637.0,2023,E21D,E21D,1,3,3,6,48,290,290,580,0.917241,1


In [46]:
# merge duplicate and non-duplicate tables together
finished_df = pd.concat([final_df_duplicates, final_df_no_duplicates], ignore_index=True)
finished_df

,patent_id,citation_pair_1,citation_pair_2,gyear,cpc_subclass_x,cpc_subclass_y,citation_pair_count_by_year,cpc_count_by_year_x,cpc_count_by_year_y,either_x_or_y_by_year,cumulative_pair_count,cumulative_total_count_x,cumulative_total_count_y,either_x_or_y_cumulative,tech_prop,revised_tech_prop
0,3988554,3940583.0,3941962.0,1976,H01H,H01H,1,2,2,4,1,2,2,4,0.750000,1.000000
1,4079218,3940583.0,3975602.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1.000000
2,4071835,3974468.0,3979704.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1.000000
3,4075587,3942142.0,3953813.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1.000000
4,4086549,4001652.0,4034266.0,1978,H01H,H01H,5,12,12,24,6,14,14,28,0.785714,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412517,11604054,4869171.0,8967048.0,2023,F42D,F42C,1,8,7,15,4,133,166,299,0.986622,0.986622
2412518,11604054,4869171.0,8967048.0,2023,F42B,F42C,1,32,7,39,7,946,166,1112,0.993705,0.993705
2412519,11472318,4378946.0,5553914.0,2022,B62B,A47D,1,97,6,103,3,837,193,1030,0.997087,0.997087
2412520,11543132,6938424.0,7779637.0,2023,F23R,F23M,1,25,4,29,12,1011,296,1307,0.990819,0.990819


In [47]:
# write to csv file
# tech prop at patent pair year level
finished_df.to_csv("finished_df.csv")


In [41]:
sort_smallest = finished_df.sort_values(by="revised_tech_prop")
sort_smallest

,patent_id,citation_pair_1,citation_pair_2,gyear,cpc_subclass_x,cpc_subclass_y,citation_pair_count_by_year,cpc_count_by_year_x,cpc_count_by_year_y,either_x_or_y_by_year,cumulative_pair_count,cumulative_total_count_x,cumulative_total_count_y,either_x_or_y_cumulative,tech_prop,revised_tech_prop
914086,4070431,3940955.0,3996321.0,1978,D06B,D01D,1,3,3,6,1,3,3,6,0.833333,0.833333
2069693,4236601,3999627.0,4079812.0,1980,E06C,A62B,1,3,4,7,1,3,4,7,0.857143,0.857143
2005046,4165961,4009988.0,4081233.0,1979,F23D,B05B,1,3,4,7,1,3,4,7,0.857143,0.857143
912329,4122401,3992678.0,4021701.0,1978,H02H,G05F,1,3,4,7,1,3,4,7,0.857143,0.857143
655195,4012704,3936777.0,3940709.0,1977,H03H,H04R,1,4,3,7,1,4,3,7,0.857143,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322345,9760182,5946274.0,8896526.0,2017,G04G,G04G,6,145,145,290,90,1240,1240,2480,0.963710,1.000000
322344,9672000,8564421.0,9257019.0,2017,G04G,G04G,6,145,145,290,90,1240,1240,2480,0.963710,1.000000
322343,9635495,8725842.0,8854925.0,2017,G04G,G04G,6,145,145,290,90,1240,1240,2480,0.963710,1.000000
322341,9846240,4899117.0,5274545.0,2017,G04G,G04G,6,145,145,290,90,1240,1240,2480,0.963710,1.000000


In [36]:
sorted_df =finished_df.sort_values(by=["cpc_subclass_x", "gyear", "cpc_subclass_y"])
first_10000 = sorted_df[10000:20000]
first_10000.to_csv("first_10000.csv")


In [40]:
# tech prop at cpc class level
finished_df_cpc_level =finished_df.drop(columns=["patent_id",	"citation_pair_1",	"citation_pair_2"])
finished_df_cpc_level.drop_duplicates(inplace=True)
finished_df_cpc_level

sorted_df1 =finished_df_cpc_level.sort_values(by=["cpc_subclass_x", "gyear", "cpc_subclass_y"])
first_10000_cpc_level = sorted_df1[10000:20000]
first_10000_cpc_level
first_10000_cpc_level.to_csv("first_10000_cpc_level.csv")